In [1]:
import csv
import lightgbm as lgb
import itertools, logging, joblib
from sklearn.model_selection import GridSearchCV
NOTE_NAME = "lgbm1"

In [2]:
ROUND = "237"
TOURNAMENT_NAME = "kazutsugi"
TARGET_NAME = f"target_{TOURNAMENT_NAME}"
PREDICTION_NAME = f"prediction_{TOURNAMENT_NAME}"

In [3]:
# Load Dataset
def read_csv(file_path):
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
    dtypes = {x: np.float16 for x in column_names if x.startswith(('feature', 'target'))}
    df = pd.read_csv(file_path, dtype=dtypes, index_col=0)
    return df

train = read_csv("../../dataset/numerai_dataset_237/numerai_training_data.csv")
tournament = read_csv("../../dataset/numerai_dataset_237/numerai_tournament_data.csv")

# reform
train["eraNumber"] = train.era.str.slice(3).astype(int)
tournament["eraNumber"] = tournament.era.str.slice(3)
train.sort_index(axis=1, inplace=True)
tournament.sort_index(axis=1, inplace=True)
cols_feature = train.columns[train.columns.str.contains("feature_")]
cols_target  = train.columns[train.columns.str.contains("target_")]

In [4]:
train.head(1).append(tournament.tail(1))

,data_type,era,eraNumber,feature_charisma1,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma2,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,feature_charisma27,feature_charisma28,feature_charisma29,feature_charisma3,feature_charisma30,feature_charisma31,feature_charisma32,feature_charisma33,feature_charisma34,feature_charisma35,feature_charisma36,feature_charisma37,feature_charisma38,feature_charisma39,feature_charisma4,feature_charisma40,feature_charisma41,feature_charisma42,feature_charisma43,feature_charisma44,feature_charisma45,feature_charisma46,feature_charisma47,feature_charisma48,feature_charisma49,feature_charisma5,feature_charisma50,feature_charisma51,feature_charisma52,feature_charisma53,feature_charisma54,feature_charisma55,feature_charisma56,feature_charisma57,feature_charisma58,feature_charisma59,feature_charisma6,feature_charisma60,feature_charisma61,feature_charisma62,feature_charisma63,feature_charisma64,feature_charisma65,feature_charisma66,feature_charisma67,feature_charisma68,feature_charisma69,feature_charisma7,feature_charisma70,feature_charisma71,feature_charisma72,feature_charisma73,feature_charisma74,feature_charisma75,feature_charisma76,feature_charisma77,feature_charisma78,feature_charisma79,feature_charisma8,feature_charisma80,feature_charisma81,feature_charisma82,feature_charisma83,feature_charisma84,feature_charisma85,feature_charisma86,feature_charisma9,feature_constitution1,feature_constitution10,feature_constitution100,feature_constitution101,feature_constitution102,feature_constitution103,feature_constitution104,feature_constitution105,feature_constitution106,feature_constitution107,feature_constitution108,feature_constitution109,feature_constitution11,feature_constitution110,feature_constitution111,feature_constitution112,feature_constitution113,feature_constitution114,feature_constitution12,feature_constitution13,feature_constitution14,feature_constitution15,feature_constitution16,feature_constitution17,feature_constitution18,feature_constitution19,feature_constitution2,feature_constitution20,feature_constitution21,feature_constitution22,feature_constitution23,feature_constitution24,feature_constitution25,feature_constitution26,feature_constitution27,feature_constitution28,feature_constitution29,feature_constitution3,feature_constitution30,feature_constitution31,feature_constitution32,feature_constitution33,feature_constitution34,feature_constitution35,feature_constitution36,feature_constitution37,feature_constitution38,feature_constitution39,feature_constitution4,feature_constitution40,feature_constitution41,feature_constitution42,feature_constitution43,feature_constitution44,feature_constitution45,feature_constitution46,feature_constitution47,feature_constitution48,feature_constitution49,feature_constitution5,feature_constitution50,feature_constitution51,feature_constitution52,feature_constitution53,feature_constitution54,feature_constitution55,feature_constitution56,feature_constitution57,feature_constitution58,feature_constitution59,feature_constitution6,feature_constitution60,feature_constitution61,feature_constitution62,feature_constitution63,feature_constitution64,feature_constitution65,feature_constitution66,feature_constitution67,feature_constitution68,feature_constitution69,feature_constitution7,feature_constitution70,feature_constitution71,feature_constitution72,feature_constitution73,feature_constitution74,feature_constitution75,feature_constitution76,feature_constitution77,feature_constitution78,feature_constitution79,feature_constitution8,feature_constitution80,feature_constitution81,feature_constitution82,feature_constitution83,feature_constitution84,feature_constitution85,feature_constitution86,feature_constitution87,feature_constitution

In [5]:
# set logger
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(f'{NOTE_NAME}/train.log')
fh.setFormatter(logging.Formatter('%(asctime)s|%(message)s', '%Y-%m-%d %H:%M:%S'))
logger.addHandler(fh)

eraは120あり、それぞれに約5000銘柄情報

---

In [6]:
# Callback for logging
from lightgbm.callback import _format_eval_result
def log_evaluation(logger, period=1, show_stdv=True, level=logging.DEBUG):
    def _callback(env):
        if period > 0 and env.evaluation_result_list and (env.iteration + 1) % period == 0:
            result = '\t'.join([_format_eval_result(x, show_stdv) for x in env.evaluation_result_list])
            logger.log(level, '[{}]\t{}'.format(env.iteration+1, result))
    _callback.order = 10
    return _callback

In [7]:
# Callback for eval records
from scipy.stats import spearmanr
class LGBMEvalHistory:
    def __init__(self, X, y, eras=None):
        self.X = X
        self.y = y
        self.eras = eras if eras is not None else np.zeros_like(y)
        self.exposure = pd.DataFrame(columns=X.columns)
        self.corr_by_era = pd.DataFrame(columns=np.unique(eras))
        self.others = pd.DataFrame()  # 後で好きに追加
        
    def calcFeatureExposure(self, pred, rounds):
        # pred: np.array
        exposures = [spearmanr(pred, self.X[col])[0] for col in self.X.columns]
        return pd.DataFrame([exposures], columns=self.X.columns, index=[rounds])
    def calcCorrByEra(self, pred, rounds):
        # pred: np.array
        res = pd.DataFrame(index=[rounds], columns=np.unique(self.eras))
        for era in np.unique(self.eras):
            n = (self.eras==era).sum()
            score = np.corrcoef(
                np.argsort(np.argsort(pred[self.eras==era])) / n,
                self.y[self.eras==era].values.reshape(-1,)
            )[0, 1]
            res.loc[rounds, era] = score
        return res  # pd.DataFrame
        
    def callback(self, env):
        pred = env.model.predict(self.X)
        self.exposure = self.exposure.append(self.calcFeatureExposure(pred, env.iteration))
        self.corr_by_era = self.corr_by_era.append(self.calcCorrByEra(pred, env.iteration))
        self.others.loc[env.iteration, "mse"] = np.mean(np.square(self.y.values.reshape(-1,) - pred.reshape(-1,)))

In [8]:
## params
search_params = {
    'n_estimators'      : [1000],
    'learning_rate'     : [0.05],
    'max_depth'         : [3, 5, 10, 30],
    'subsample'         : [0.1, 0.5, 1.0],
    'reg_alpha'         : [0.1, 0.5, 1.0]
}

In [ ]:
## GridSearch
i = 0;
for val in itertools.product(*search_params.values()):
    param_dict = dict(zip(search_params.keys(), val))
    logger.info(param_dict)
    
    # Fit
    model = lgb.LGBMRegressor(silent=True, early_stopping_rounds=30, random_state=1, **param_dict)
    hist_train = LGBMEvalHistory(train.loc[:, cols_feature],
                                 train.loc[:, cols_target],
                                 eras=train.era)
    hist_tournament = LGBMEvalHistory(tournament.query("data_type=='validation'").loc[:, cols_feature],
                                      tournament.query("data_type=='validation'").loc[:, cols_target],
                                      eras=tournament.query("data_type=='validation'").era)
    model.fit(X=train.loc[:, cols_feature], y=train.loc[:, cols_target], 
              eval_set=(tournament.query("data_type=='validation'").loc[:, cols_feature],
                        tournament.query("data_type=='validation'").loc[:, cols_target]),
              callbacks=[hist_train.callback, hist_tournament.callback, log_evaluation(logger)],
              verbose=False)
    
    # save model, save log
    joblib.dump(model, f"{NOTE_NAME}/lgbReg_{i}.pkl")
    hist_train.corr_by_era.to_hdf(f"{NOTE_NAME}/corr_by_era_{i}.hdf", "pandasdf")
    hist_train.exposure.to_hdf(f"{NOTE_NAME}/exposures_{i}.hdf", "pandasdf")
    hist_train.others.to_hdf(f"{NOTE_NAME}/metrics_{i}.hdf", "pandasdf")
    hist_tournament.corr_by_era.to_hdf(f"{NOTE_NAME}/val_corr_by_era_{i}.hdf", "pandasdf")
    hist_tournament.exposure.to_hdf(f"{NOTE_NAME}/val_exposures_{i}.hdf", "pandasdf")
    hist_tournament.others.to_hdf(f"{NOTE_NAME}/val_metrics_{i}.hdf", "pandasdf")
    logger.info("save completed.")
    
    i += 1

/home/tarai/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
